In [3]:
import pandas as pd
from matplotlib import pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
import numpy as np
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from math import sqrt
import matplotlib.pyplot as plt
from mlxtend.plotting import scatterplotmatrix
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso, LassoCV
import seaborn as sns
from matplotlib import pyplot as plt

In [4]:

df_0 = pd.read_csv('dataset02-RED-24hs.csv')
df_2=pd.read_csv('dataset03-RED-24hs.csv')


In [6]:
df_0.columns.values

array(['irq_0', 'skew_0', 'kurtosis_0', 'f0_0', 'Pf0_0', 'zeroX_0',
       'p2p_0', 'crest_0', 'irq_1', 'skew_1', 'kurtosis_1', 'f0_1',
       'Pf0_1', 'zeroX_1', 'p2p_1', 'crest_1', 'irq_2', 'skew_2',
       'kurtosis_2', 'f0_2', 'Pf0_2', 'zeroX_2', 'p2p_2', 'crest_2',
       'irq_3', 'skew_3', 'kurtosis_3', 'f0_3', 'Pf0_3', 'zeroX_3',
       'p2p_3', 'crest_3', 'covar_01', 'covar_02', 'covar_03', 'covar_12',
       'covar_13', 'covar_23', 'y0', 'y1', 'y2', 'y3'], dtype=object)

In [8]:
df_2.columns.values

array(['irq_0', 'skew_0', 'kurtosis_0', 'f0_0', 'Pf0_0', 'zeroX_0',
       'p2p_0', 'crest_0', 'irq_1', 'skew_1', 'kurtosis_1', 'f0_1',
       'Pf0_1', 'zeroX_1', 'p2p_1', 'crest_1', 'irq_2', 'skew_2',
       'kurtosis_2', 'f0_2', 'Pf0_2', 'zeroX_2', 'p2p_2', 'crest_2',
       'irq_3', 'skew_3', 'kurtosis_3', 'f0_3', 'Pf0_3', 'zeroX_3',
       'p2p_3', 'crest_3', 'covar_01', 'covar_02', 'covar_03', 'covar_12',
       'covar_13', 'covar_23', 'y0', 'y1', 'y2', 'y3'], dtype=object)

Creamos dos nuevos dataframes para hacer predicciones sobre el rodamiento del cual tenemos la informacion que fue el primero en fallar, en este caso para un conjunto de datos fallo b0 (usado para train) y para el otro b2 (usado para test)

In [10]:
new_features2=['irq_2', 'skew_2', 'kurtosis_2', 'f0_2', 'Pf0_2',
       'zeroX_2', 'p2p_2', 'crest_2', 'covar_02',
       'covar_12','covar_23']

In [11]:
new_features0=['irq_0', 'skew_0', 'kurtosis_0', 'f0_0', 'Pf0_0', 'zeroX_0', 'p2p_0',
       'crest_0','covar_01', 'covar_02',
       'covar_03'] 


In [12]:
df_2_tr=df_2[new_features2]
df_0_tr=df_0[new_features0]


In [13]:
df_0_tr.head()

,irq_0,skew_0,kurtosis_0,f0_0,Pf0_0,zeroX_0,p2p_0,crest_0,covar_01,covar_02,covar_03
0,0.096,0.083993,0.628763,985.351562,52.733122,6593.0,0.840,6.120331,0.001398,-0.000501,-0.000304
1,0.097,0.052142,0.648291,985.351562,51.201049,6867.0,0.757,4.895038,0.001287,-0.000289,-0.000188
2,0.100,0.032808,0.513475,985.351562,50.779361,6637.0,0.903,6.598472,0.001349,-0.000503,-0.000284
3,0.100,0.041486,1.157953,985.351562,51.079815,6659.0,1.184,7.723217,0.001410,-0.000455,-0.000221
4,0.103,0.028224,0.603177,985.351562,50.279226,6765.0,0.782,4.982524,0.001668,-0.000510,-0.000310


In [24]:
df_2_tr.head()

,irq_2,skew_2,kurtosis_2,f0_2,Pf0_2,zeroX_2,p2p_2,crest_2,covar_02,covar_12,covar_23
0,0.086,0.083455,0.592253,984.375000,45.123977,7990.0,0.671,5.437565,-0.000123,-0.000131,0.000086
1,0.086,0.011829,0.360079,985.351562,45.995787,8024.0,0.590,4.450449,-0.000138,-0.000254,0.000128
2,0.093,0.049060,0.214038,231.445312,47.827940,7895.0,0.572,4.133782,-0.000110,-0.000303,0.000162
3,0.093,0.002209,0.299358,231.445312,45.885568,7831.0,0.659,5.084751,-0.000261,-0.000360,0.000235
4,0.088,0.031886,0.396086,986.328125,45.616624,7928.0,0.762,5.750781,-0.000290,-0.000473,0.000248


In [23]:
#for i in df_0_tr.columns:
 # plot_acf(df_0_tr[i],alpha=0.01,title=i)

Transformacion del dataset

In [16]:
from pandas import DataFrame
from pandas import concat
 
def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]
	# put it all together
	agg = concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg
 


In [18]:
df_tr0 = series_to_supervised(df_0_tr, 10)
df_tr2 = series_to_supervised(df_2_tr, 10)


In [21]:
target2=df_2['y2'][10:]
target0=df_0['y0'][10:]

In [22]:
df_out_2 = pd.concat([df_tr2, target2], axis=1)
df_out_0 = pd.concat([df_tr0, target0], axis=1)

In [26]:
# Save to CSV
PATH_DATASET = "./datasetTEST-24hs.csv"
df_out_2.to_csv(PATH_DATASET, index=False)

In [27]:
PATH_DATASET = "./datasetTRAIN-24hs.csv"
df_out_0.to_csv(PATH_DATASET, index=False)